# 8장 불변 특징의 표현과 정합 - Images stitching

# ▶ 이미지 스티칭
- 사진 이어 붙이기, 파노라마 영상(panorama image)
- 방법
    - 여러 장의 영상에서 특징점을 검출
    - 동일한 특징점 찾기(매칭)
    - 투시변환을 이용하여 붙이기(영상 결합)
    - 노출 보정, 블렌딩을 적용

# ▶ OpenCV
- Images stitching
    - https://docs.opencv.org/4.x/d1/d46/group_stitching.html
- Stitcher Class Reference
    - cv2.Stitcher_create() -> retval
    - cv.Stitcher.stitch(images) -> retval, pano
        - images: Input images.

In [1]:
import sys
import os
import numpy as np
import cv2

In [2]:
### change the current working directory
os.chdir(r'C:\Users\sse88\Downloads')

### 입력 영상 이름
img_names = ['hnu_20220530_01.jpg', 'hnu_20220530_02.jpg']

### 영상 저장
imgs = []
for name in img_names:
    img = cv2.imread(name)
    
    if img is None:
        print('Image load failed!')
        sys.exit()
    
    imgs.append(img)

### 이미지 스티칭
stitcher = cv2.Stitcher_create()
status, dst = stitcher.stitch(imgs)

if status != cv2.Stitcher_OK:
    print('Stitch failed!')
    sys.exit()

cv2.imwrite('output.jpg', dst)

cv2.namedWindow('dst', cv2.WINDOW_NORMAL)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

# ▶ 파노라마 사진 생성기
- https://blog.naver.com/PostView.naver?blogld=teach3450&logNo=221983420407
- 힌트
    - 따로 찍은 2장의 사진 중에 좌측과 우측에 연결한 순서를 정한다.
    - 각각 특징점과 디스크립터를 구해 매칭기로 우측사진을 기준으로 좌측사진을 매칭한다.
    - 원근 변환행렬을 구해 cv2.warpPerspective() 함수로 원근변환한다.
    - 결과 영상 크기를 지정할 때 2개의 영상을 더한 크기를 지정해 그 결과에 좌측 원본사진을 합성한다.

In [ ]:
!pip install opencv-contrib-python

In [3]:
import numpy as np, cv2
# 왼쪽/오른쪽 사진 읽기
# 왼쪽 사진 = train = 매칭의 대상
imgL = cv2.imread("hnu_20220530_01.jpg")
# 오른쪽 사진 = query = 매칭의 기준
imgR = cv2.imread("hnu_20220530_02.jpg")
hl, wl = imgL.shape[:2] # 왼쪽 사진 높이, 넓이
hr, wr = imgR.shape[:2] # 오른쪽 사진 높이, 넓이
grayL = cv2.cvtColor(imgL, cv2.COLOR_BGR2GRAY) # 그레이 스케일 변환
grayR = cv2.cvtColor(imgR, cv2.COLOR_BGR2GRAY) # 그레이 스케일 변환

# SIFT 특징 검출기 생성 및 특징점 검출
descriptor = cv2.xfeatures2d.SIFT_create() # SIFT 추출기 생성
(kpsL, featuresL) = descriptor.detectAndCompute(imgL, None) # 키포인트, 디스크립터
(kpsR, featuresR) = descriptor.detectAndCompute(imgR, None) # 키포이트, 디스크립터

# BF 매칭기 생성 및 knn 매칭
matcher = cv2.DescriptorMatcher_create("BruteForce") # BF 매칭기 생성
matches = matcher.knnMatch(featuresR, featuresL, 2) # knn 매칭

# 좋은 매칭점 선별
good_matches = []
for m in matches:
    if len(m) == 2 and m[0].distance < m[1].distance * 0.75: ### 75%
        good_matches.append((m[0].trainIdx, m[0].queryIdx))
print('matches:{}/{}'.format(len(good_matches), len(matches)))

# 좋은 매칭점이 4개 이상인 원근 변환행렬 구하기
if len(good_matches) > 4:
    ptsL = np.float32([kpsL[i].pt for (i,_) in good_matches]) # 좋은 매칭점 좌표
    ptsR = np.float32([kpsR[i].pt for (_,i) in good_matches]) # 좋은 매칭점 좌표
    mtrx, status = cv2.findHomography(ptsR, ptsL, cv2.RANSAC, 4.0)
    # 원근 변환행렬로 오른쪽 사진을 원근 변환, 결과 이미지 크기는 사진 2장 크기
    panorama = cv2.warpPerspective(imgR, mtrx, (wr + wl, hr))
    # 왼쪽 사진을 원근 변환한 왼쪽 영역에 합성
    panorama[0:hl, 0:wl] = imgL
else: # 좋은 매칭점이 4개가 안 되는 경우
    panorama = imgL

# 결과 출력
cv2.imshow('Left Image', imgL)
cv2.imshow('Right Image', imgR)
cv2.imshow('Panorama', panorama)
cv2.waitKey(0)
cv2.destroyWindow()

AttributeError: module 'cv2' has no attribute 'xfeatures2d'